In [1]:
## load data 

import numpy as np
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
import itertools
from tensorflow.keras.utils import to_categorical
from src.score import *
import re
from collections import OrderedDict


DATADIR = '/rds/general/user/mc4117/home/WeatherBench/data/'

In [2]:
var_dict = {
    'geopotential': ('z', [500]),
    'temperature': ('t', [850])}   

ds_list = []

for long_var, params in var_dict.items():
    if long_var == 'constants':
        ds_list.append(xr.open_mfdataset(f'{DATADIR}/{long_var}/*.nc', combine='by_coords'))
    else:
        var, levels = params
        if levels is not None:
            ds_list.append(xr.open_mfdataset(f'{DATADIR}/{long_var}/*.nc', combine='by_coords').sel(level = levels))
        else:
            ds_list.append(xr.open_mfdataset(f'{DATADIR}/{long_var}/*.nc', combine='by_coords'))

# because missing first values of solar radiation exclude these from the dataset
ds_whole = xr.merge(ds_list).isel(time = slice(7, None))

ds_train = ds_whole.sel(time=slice('1979', '2016'))  
ds_test = ds_whole.sel(time=slice('2017', '2018'))
ds_valid = ds_whole.sel(time=slice('2015', '2016'))

class DataGenerator(keras.utils.Sequence):
    def __init__(self, ds, var_dict, lead_time, batch_size=32, shuffle=True, load=True, 
                 mean=None, std=None, output_vars= None, bins_z = None):
        """
        Data generator for WeatherBench data.
        Template from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
        Args:
            ds: Dataset containing all variables
            var_dict: Dictionary of the form {'var': level}. Use None for level if data is of single level
            lead_time: Lead time in hours
            batch_size: Batch size
            shuffle: bool. If True, data is shuffled.
            load: bool. If True, datadet is loaded into RAM.
            mean: If None, compute mean from data.
            std: If None, compute standard deviation from data.
        """
        
        self.ds = ds
        self.var_dict = var_dict
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.lead_time = lead_time

        data = []
        level_names = []
        generic_level = xr.DataArray([1], coords={'level': [1]}, dims=['level'])
        for long_var, params in var_dict.items():
            if long_var == 'constants': 
                for var in params:
                    data.append(ds[var].expand_dims(
                        {'level': generic_level, 'time': ds.time}, (1, 0)
                    ))
                    level_names.append(var)
            else:
                var, levels = params
                try:
                    data.append(ds[var].sel(level=levels))
                    level_names += [f'{var}_{level}' for level in levels]
                except ValueError:
                    data.append(ds[var].expand_dims({'level': generic_level}, 1))
                    level_names.append(var)   

        self.data = xr.concat(data, 'level').transpose('time', 'lat', 'lon', 'level')
        self.data['level_names'] = xr.DataArray(
            level_names, dims=['level'], coords={'level': self.data.level})
        if output_vars is None:
            self.output_idxs = range(len(ds.data.level))
        else:
            self.output_idxs = [i for i, l in enumerate(self.data.level_names.values) 
                                if any([bool(re.match(o, l)) for o in output_vars])]

        self.bins_z = np.linspace(self.data.isel(level =self.output_idxs).min(), self.data.isel(level =self.output_idxs).max(), 100) if bins_z is None else bins_z 
        self.binned_data = xr.DataArray((np.digitize(self.data.isel(level=self.output_idxs), self.bins_z)-1)[:,:,:,0], dims=['time', 'lat', 'lon'], coords={'time':self.data.time.values, 'lat': self.data.lat.values, 'lon': self.data.lon.values})
        
        self.mean = self.data.mean(('time', 'lat', 'lon')).compute() if mean is None else mean
        self.std = self.data.std(('time', 'lat', 'lon')).compute() if std is None else std
        # Normalize
        self.data = (self.data - self.mean) / self.std
        self.n_samples = self.data.isel(time=slice(0, -lead_time)).shape[0]
        self.init_time = self.data.isel(time=slice(None, -lead_time)).time
        self.valid_time = self.data.isel(time=slice(lead_time, None)).time

        del ds
        self.on_epoch_end()

        # For some weird reason calling .load() earlier messes up the mean and std computations
        if load: print('Loading data into RAM'); self.data.load()
        if load: print('Loading data into RAM'); self.binned_data.load()            

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.n_samples / self.batch_size))

    def __getitem__(self, i):
        'Generate one batch of data'
        idxs = self.idxs[i * self.batch_size:(i + 1) * self.batch_size]
        X = self.data.isel(time=idxs).values
        y = self.binned_data.isel(time=idxs + self.lead_time).values
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.idxs = np.arange(self.n_samples)
        if self.shuffle == True:
            np.random.shuffle(self.idxs)

bs=32
lead_time=72
output_vars = ['z_500']

# Create a training and validation data generator. Use the train mean and std for validation as well.
dg_train = DataGenerator(
    ds_train.sel(time=slice('1979', '2013')), var_dict, lead_time, batch_size=bs, load=False, output_vars = output_vars)

dg_valid = DataGenerator(
    ds_train.sel(time=slice('2015', '2016')), var_dict, lead_time, batch_size=bs, mean=dg_train.mean, std=dg_train.std, bins_z = dg_train.bins_z, shuffle=False, output_vars = output_vars)

"""
dg_valid2 = DataGenerator(
    ds_train.sel(time=slice('2014', '2014')), var_dict, lead_time, batch_size=bs, mean=dg_train.mean, std=dg_train.std, bins_z = dg_train.bins_z, shuffle=False, output_vars = output_vars)
"""
dg_test = DataGenerator(
    ds_test, var_dict, lead_time, batch_size=bs, mean=dg_train.mean, std=dg_train.std, bins_z = dg_train.bins_z, shuffle=False, output_vars = output_vars)


Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM


In [3]:
block_no = 9

In [4]:
del dg_train

In [5]:
output_avg_geo = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_geo_[300, 400, 500, 600, 700, 850]_preds_cat_val.npy')
fc_conv_ds_avg = xr.Dataset({
        'z': xr.DataArray(
              np.dot(output_avg_geo, dg_valid.bins_z),
               dims=['time', 'lat', 'lon'],
               coords={'time':dg_valid.data.time[72:], 'lat': dg_valid.data.lat, 'lon': dg_valid.data.lon,
                })})
    
cnn_rmse_arg = compute_weighted_rmse(fc_conv_ds_avg, ds_valid.z.sel(level = 500)[72:]).compute()
cnn_rmse_arg

ValueError: shapes (43776,32,64) and (100,) not aligned: 64 (dim 2) != 100 (dim 0)

In [13]:
output_avg_temp = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_temp_[300, 400, 500, 600, 700, 850]_preds_cat_val.npy')

fc_conv_ds_avg = xr.Dataset({
        'z': xr.DataArray(
              np.dot(output_avg_temp, dg_valid.bins_z),
               dims=['time', 'lat', 'lon'],
               coords={'time':dg_valid.data.time[72:], 'lat': dg_valid.data.lat, 'lon': dg_valid.data.lon,
                })})
    
cnn_rmse_arg = compute_weighted_rmse(fc_conv_ds_avg, ds_valid.z.sel(level = 500)[72:]).compute()
cnn_rmse_arg

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int64 500
Data variables:
    z        float64 415.9

In [17]:
del output_avg_geo

In [18]:
output_avg_pv = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_pot_vort_[150, 250, 300, 700, 850]_preds_cat_val.npy')

fc_conv_ds_avg = xr.Dataset({
        'z': xr.DataArray(
              np.dot(output_avg_pv, dg_valid.bins_z),
               dims=['time', 'lat', 'lon'],
               coords={'time':dg_valid.data.time[72:], 'lat': dg_valid.data.lat, 'lon': dg_valid.data.lon,
                })})
    
cnn_rmse_arg = compute_weighted_rmse(fc_conv_ds_avg, ds_valid.z.sel(level = 500)[72:]).compute()
cnn_rmse_arg

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int64 500
Data variables:
    z        float64 430.5

In [12]:
output_avg_sh = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_specific_humidity_[150, 200, 600, 700, 850, 925, 1000]_preds_cat_val.npy')

fc_conv_ds_avg = xr.Dataset({
        'z': xr.DataArray(
              np.dot(output_avg_sh, dg_valid.bins_z),
               dims=['time', 'lat', 'lon'],
               coords={'time':dg_valid.data.time[72:], 'lat': dg_valid.data.lat, 'lon': dg_valid.data.lon,
                })})
    
cnn_rmse_arg = compute_weighted_rmse(fc_conv_ds_avg, ds_valid.z.sel(level = 500)[72:]).compute()
cnn_rmse_arg

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int64 500
Data variables:
    z        float64 438.6

In [19]:
output_avg_sr = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_solar_rad_no_l_preds_cat_val.npy')

fc_conv_ds_avg = xr.Dataset({
        'z': xr.DataArray(
              np.dot(output_avg_sr, dg_valid.bins_z),
               dims=['time', 'lat', 'lon'],
               coords={'time':dg_valid.data.time[72:], 'lat': dg_valid.data.lat, 'lon': dg_valid.data.lon,
                })})
    
cnn_rmse_arg = compute_weighted_rmse(fc_conv_ds_avg, ds_valid.z.sel(level = 500)[72:]).compute()
cnn_rmse_arg

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int64 500
Data variables:
    z        float64 460.4

In [26]:
output_avg_const = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_const_no_l_preds_cat_val.npy')

fc_conv_ds_avg = xr.Dataset({
        'z': xr.DataArray(
              np.dot(output_avg_const, dg_valid.bins_z),
               dims=['time', 'lat', 'lon'],
               coords={'time':dg_valid.data.time[72:], 'lat': dg_valid.data.lat, 'lon': dg_valid.data.lon,
                })})
    
cnn_rmse_arg = compute_weighted_rmse(fc_conv_ds_avg, ds_valid.z.sel(level = 500)[72:]).compute()
cnn_rmse_arg

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int64 500
Data variables:
    z        float64 458.3

In [5]:
output_avg_geo = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_geo_[300, 400, 500, 600, 700, 850]_preds_cat_val.npy')
output_avg_temp = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_temp_[300, 400, 500, 600, 700, 850]_preds_cat_val.npy')
output_avg_pv = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_pot_vort_[150, 250, 300, 700, 850]_preds_cat_val.npy')
output_avg_sh = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_specific_humidity_[150, 200, 600, 700, 850, 925, 1000]_preds_cat_val.npy')
#output_avg_sr = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_solar_rad_no_l_preds_cat_val.npy')
output_avg_const = np.load('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/' + str(block_no) + '_const_no_l_preds_cat_val.npy')

output_sum = (output_avg_geo + output_avg_temp + output_avg_pv + output_avg_sh + output_avg_const)/5

MemoryError: Unable to allocate 26.7 GiB for an array with shape (17472, 32, 64, 100) and data type float64

In [6]:
del output_avg_geo
del output_avg_temp

In [7]:
fc_conv_ds_avg = xr.Dataset({
        'z': xr.DataArray(
              np.dot(output_sum, dg_valid.bins_z),
               dims=['time', 'lat', 'lon'],
               coords={'time':dg_valid.data.time[72:], 'lat': dg_valid.data.lat, 'lon': dg_valid.data.lon,
                })})
    
cnn_rmse_arg = compute_weighted_rmse(fc_conv_ds_avg, ds_valid.z.sel(level = 500)[72:]).compute()
cnn_rmse_arg

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    level    int64 500
Data variables:
    z        float64 412.4

In [6]:
sh_test = xr.open_dataset('/rds/general/user/mc4117/home/WeatherBench/saved_pred_data/9_specific_humidity_[300, 500, 600, 700, 850, 925, 1000]_preds_newtest.nc')

In [32]:
(np.dot(output_avg_geo, dg_valid.bins_z)-float(dg_valid.mean.sel(level = 500)))/float(dg_valid.std.sel(level = 500))

array([[[-0.93838274, -0.93541296, -0.9353863 , ..., -0.94185169,
         -0.94139868, -0.93772298],
        [-0.95007743, -0.93375232, -0.91958038, ..., -1.00582205,
         -0.98630252, -0.96814844],
        [-1.02387479, -1.00285162, -0.99034787, ..., -1.10885472,
         -1.08144982, -1.05387639],
        ...,
        [-1.04570575, -1.07327253, -1.09797437, ..., -0.98692293,
         -1.00151355, -1.02642516],
        [-1.13091652, -1.14184192, -1.16083395, ..., -1.12377288,
         -1.13001979, -1.12701784],
        [-1.17504872, -1.17552432, -1.19179021, ..., -1.1707214 ,
         -1.17190232, -1.17069475]],

       [[-0.92990912, -0.93213695, -0.93131579, ..., -0.93737881,
         -0.93725679, -0.93748446],
        [-0.95396691, -0.93618427, -0.92830773, ..., -1.01305939,
         -0.99255115, -0.97116873],
        [-1.03209083, -1.01294766, -1.00601537, ..., -1.10722345,
         -1.08519954, -1.06117164],
        ...,
        [-1.05137894, -1.0733375 , -1.10552817, ..., -